In [1]:
from context_analyser import *

In [2]:
CA = ContextAnalyser()

In [3]:
X, Y = CA.getXY()

In [4]:
CA.get_context(Y[10].argmax())

'zwierzęta'

In [5]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
Y.argmax(axis=1)

array([ 1,  6,  5,  8, 12, 11, 12,  7, 10,  0,  2,  2,  2,  3,  3,  4,  4,
        4,  9,  9], dtype=int64)

In [7]:
from sklearn.neural_network import MLPClassifier

C:\Programs\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 5), random_state=1, max_iter=100)
clf.fit(X, Y)

C:\Programs\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 5), max_iter=100,
              random_state=1, solver='lbfgs')

In [14]:
for i in range(CA.get_len()):
    pred_idx = clf.predict_proba([X[i]])[0].argmax()
    org_idx = Y[i].argmax()
    print("pred:{}, oryg:{}".format(CA.get_context(pred_idx), CA.get_context(org_idx)))

pred:markiza, oryg:markiza
pred:trasa, oryg:whiskey
pred:trasa, oryg:trasa
pred:szczepienia, oryg:szczepienia
pred:wypadek, oryg:wypadek
pred:trasa, oryg:ropa
pred:markiza, oryg:wypadek
pred:sondaż, oryg:sondaż
pred:wsparcie, oryg:wsparcie
pred:banan, oryg:banan
pred:zwierzęta, oryg:zwierzęta
pred:zwierzęta, oryg:zwierzęta
pred:zwierzęta, oryg:zwierzęta
pred:sport, oryg:sport
pred:sport, oryg:sport
pred:astronomia, oryg:astronomia
pred:astronomia, oryg:astronomia
pred:astronomia, oryg:astronomia
pred:historia, oryg:historia
pred:astronomia, oryg:historia
